In [1]:
import scanpy as sc
data_dir = '../../dataset/GEX_human_neocortex_filtered.h5ad'
adata = sc.read_h5ad(data_dir)
print(adata.X)
sc.pp.normalize_total(adata,target_sum=1e4)
sc.pp.log1p(adata)

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 560546360 stored elements and shape (232328, 18944)>
  Coords	Values
  (0, 9055)	1.0
  (0, 484)	1.0
  (0, 8769)	1.0
  (0, 15607)	1.0
  (0, 5958)	1.0
  (0, 6754)	2.0
  (0, 10091)	4.0
  (0, 720)	1.0
  (0, 211)	2.0
  (0, 5468)	1.0
  (0, 9205)	1.0
  (0, 13556)	2.0
  (0, 550)	2.0
  (0, 5036)	1.0
  (0, 14257)	1.0
  (0, 16799)	1.0
  (0, 4038)	4.0
  (0, 4799)	1.0
  (0, 13032)	1.0
  (0, 1974)	2.0
  (0, 14845)	1.0
  (0, 17775)	3.0
  (0, 9728)	1.0
  (0, 17921)	1.0
  (0, 12549)	1.0
  :	:
  (232327, 13612)	1.0
  (232327, 4926)	1.0
  (232327, 647)	2.0
  (232327, 10195)	2.0
  (232327, 9526)	1.0
  (232327, 12032)	1.0
  (232327, 8830)	2.0
  (232327, 6370)	15.0
  (232327, 99)	3.0
  (232327, 14319)	1.0
  (232327, 14924)	1.0
  (232327, 5053)	7.0
  (232327, 9166)	4.0
  (232327, 18338)	2.0
  (232327, 10926)	1.0
  (232327, 14013)	2.0
  (232327, 2278)	1.0
  (232327, 14281)	1.0
  (232327, 11653)	1.0
  (232327, 11494)	7.0
  (232327, 13872)	1.0
  (232

In [2]:
import torch
import lightning.pytorch as pl
from self_supervision.models.lightning_modules.cellnet_autoencoder import MLPAutoEncoder
from self_supervision.estimator.cellnet import EstimatorAutoEncoder

# 设置你的 .ckpt 文件路径
ckpt_path = "../../sc_pretrained/Pretrained Models/GPMask.ckpt"

# 模型参数
units_encoder = [512, 512, 256, 256, 64]
units_decoder = [256, 256, 512, 512]

# 初始化 EstimatorAutoEncoder 实例
estim = EstimatorAutoEncoder(data_path=None)  # 如果没有实际数据路径，可以设置为None

# 加载预训练模型
estim.model = MLPAutoEncoder.load_from_checkpoint(
    ckpt_path,
    gene_dim=19331,  # 根据你的数据调整
    batch_size=2048,  # 根据你的需要调整
    units_encoder=units_encoder, 
    units_decoder=units_decoder,
    masking_strategy="random",  # 假设模型使用了随机掩码
    masking_rate=0.5,  # 根据需要调整
)

# 使用 GPU 进行评估（如果可用）
estim.trainer = pl.Trainer(accelerator="gpu", devices=1 if torch.cuda.is_available() else None)
estim.model

/home/hanchuangyi/miniconda3/envs/ssl/lib/python3.10/site-packages/merlin/dtypes/mappings/tf.py:52: UserWarning: Tensorflow dtype mappings did not load successfully due to an error: No module named 'tensorflow'
  warn(f"Tensorflow dtype mappings did not load successfully due to an error: {exc.msg}")
/home/hanchuangyi/miniconda3/envs/ssl/lib/python3.10/site-packages/merlin/dtypes/mappings/triton.py:53: UserWarning: Triton dtype mappings did not load successfully due to an error: No module named 'tritonclient'
  warn(f"Triton dtype mappings did not load successfully due to an error: {exc.msg}")


GPU available: True (cuda), used: True


TPU available: False, using: 0 TPU cores


HPU available: False, using: 0 HPUs


MLPAutoEncoder(
  (train_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=train_
  )
  (val_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=val_
  )
  (test_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=test_
  )
  (encoder): MLP(
    (0): Linear(in_features=19331, out_features=512, bias=True)
    (1): SELU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): SELU()
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): SELU()
    (8): Dropout(p=0.1, inplace=False)
    (9): Linear(in_features=256, out_features=256, b

In [3]:
import pandas as pd
var_df = pd.read_parquet('../../sc_pretrained/var.parquet')
var_df

,feature_id,feature_name
0,ENSG00000186092,OR4F5
1,ENSG00000284733,OR4F29
2,ENSG00000284662,OR4F16
3,ENSG00000187634,SAMD11
4,ENSG00000188976,NOC2L
...,...,...
19326,ENSG00000288702,UGT1A3
19327,ENSG00000288705,UGT1A5
19328,ENSG00000182484,WASH6P
19329,ENSG00000288622,PDCD6-AHRR


In [4]:
all_genes = var_df['feature_name'].tolist()
all_genes

['OR4F5',
 'OR4F29',
 'OR4F16',
 'SAMD11',
 'NOC2L',
 'KLHL17',
 'PLEKHN1',
 'PERM1',
 'HES4',
 'ISG15',
 'AGRN',
 'RNF223',
 'C1orf159',
 'TTLL10',
 'TNFRSF18',
 'TNFRSF4',
 'SDF4',
 'B3GALT6',
 'C1QTNF12',
 'UBE2J2',
 'SCNN1D',
 'ACAP3',
 'PUSL1',
 'INTS11',
 'CPTP',
 'TAS1R3',
 'DVL1',
 'MXRA8',
 'AURKAIP1',
 'CCNL2',
 'MRPL20',
 'ANKRD65',
 'TMEM88B',
 'VWA1',
 'ATAD3C',
 'ATAD3B',
 'ATAD3A',
 'TMEM240',
 'SSU72',
 'FNDC10',
 'MIB2',
 'MMP23B',
 'CDK11B',
 'SLC35E2B',
 'CDK11A',
 'NADK',
 'GNB1',
 'CALML6',
 'TMEM52',
 'CFAP74',
 'GABRD',
 'PRKCZ',
 'FAAP20',
 'SKI',
 'MORN1',
 'RER1',
 'PEX10',
 'PLCH2',
 'PANK4',
 'HES5',
 'TNFRSF14',
 'PRXL2B',
 'MMEL1',
 'TTC34',
 'ACTRT2',
 'PRDM16',
 'ARHGEF16',
 'MEGF6',
 'TPRG1L',
 'WRAP73',
 'TP73',
 'CCDC27',
 'SMIM1',
 'LRRC47',
 'CEP104',
 'DFFB',
 'C1orf174',
 'AJAP1',
 'NPHP4',
 'KCNAB2',
 'CHD5',
 'RPL22',
 'RNF207',
 'ICMT',
 'HES3',
 'GPR153',
 'ACOT7',
 'HES2',
 'ESPN',
 'TNFRSF25',
 'PLEKHG5',
 'NOL9',
 'TAS1R1',
 'ZBTB48',
 'KLH

In [5]:
adata.var['gene_name']=adata.var.index
adata.var['gene_name']

ATXN7L1    ATXN7L1
ZNF593      ZNF593
SLC44A1    SLC44A1
COX18        COX18
FSIP2        FSIP2
            ...   
TOX3          TOX3
BUB1          BUB1
RHNO1        RHNO1
CSTF3        CSTF3
FSD1L        FSD1L
Name: gene_name, Length: 18944, dtype: object

In [6]:
import numpy as np
# 初始化一个新的数据矩阵，形状为 (adata.X.shape[0], len(all_genes))，填充为零
new_data = np.zeros((adata.X.shape[0], len(all_genes)), dtype=np.float32)

In [7]:
existing_genes = adata.var['gene_name']
existing_genes

ATXN7L1    ATXN7L1
ZNF593      ZNF593
SLC44A1    SLC44A1
COX18        COX18
FSIP2        FSIP2
            ...   
TOX3          TOX3
BUB1          BUB1
RHNO1        RHNO1
CSTF3        CSTF3
FSD1L        FSD1L
Name: gene_name, Length: 18944, dtype: object

In [8]:
# 将所有基因名称转换为小写
all_genes_lower = [gene.lower() for gene in all_genes]
adata_genes_lower = [gene.lower() for gene in existing_genes]

# 将两个列表转换为集合
all_genes_set = set(all_genes_lower)
adata_genes_set = set(adata_genes_lower)

# 计算交集
matching_genes = all_genes_set.intersection(adata_genes_set)
matching_count = len(matching_genes)
# 计算不匹配的基因
non_matching_genes = adata_genes_set - matching_genes
non_matching_count = len(non_matching_genes)


# 输出结果
print(f"匹配的基因数量: {matching_count}")
print(f"匹配的基因列表: {matching_genes}")
non_matching_genes


匹配的基因数量: 18944
匹配的基因列表: {'zscan1', 'znf627', 'fbl', 'raet1g', 'wdr5', 'mgp', 'ak4', 'or2t33', 'cmas', 'serpina9', 'echdc1', 'chmp2b', 'adam7', 'adhfe1', 'pskh1', 'pygl', 'gpat3', 'fmo2', 'tm9sf1', 'folr1', 'spsb1', 'cyp2f1', 'ifna14', 'slc8a3', 'actg2', 'taok2', 'ubxn6', 'cacybp', 'brd3os', 'katnbl1', 'dclk3', 'srsf9', 'rtca', 'dazl', 'hgs', 'mcm6', 'timmdc1', 'crybg3', 'tmem150b', 'ecpas', 'dffb', 'adh6', 'rnasek', 'tdrd3', 'nnmt', 'ipp', 'mc2r', 'pmp22', 'ush1c', 'phtf2', 'fam3c', 'mt-co2', 'tiparp', 'prex2', 'zc3h4', 'lgals14', 'mageb16', 'tyw1b', 'mtrr', 'rgs3', 'rps26', 'manba', 'actrt1', 'ttc37', 'snx20', 'itm2b', 'u2af1l4', 'mea1', 'nup93', 'mymk', 'f11r', 'rbm17', 'mgat4d', 'gpr150', 'arhgap15', 'pnrc1', 'heg1', 'hcfc2', 'cma1', 'trrap', 'chrne', 'cdk2', 'ccn1', 'ankrd33', 'nrgn', 'rhpn2', 'adarb1', 'cks1b', 'tex101', 'miox', 'ctag1b', 'trafd1', 'ddb1', 'brca1', 'krtap19-5', 'map3k6', 'znf735', 'plce1', 'wdr87', 'apobec3a', 'asb15', 'ceacam5', 'ttk', 'mycl', 'ush1g', 'nkx6-2', 

set()

In [9]:
gene_to_index = {gene: idx for idx, gene in enumerate(all_genes_lower)}
gene_to_index

{'or4f5': 0,
 'or4f29': 1,
 'or4f16': 2,
 'samd11': 3,
 'noc2l': 4,
 'klhl17': 5,
 'plekhn1': 6,
 'perm1': 7,
 'hes4': 8,
 'isg15': 9,
 'agrn': 10,
 'rnf223': 11,
 'c1orf159': 12,
 'ttll10': 13,
 'tnfrsf18': 14,
 'tnfrsf4': 15,
 'sdf4': 16,
 'b3galt6': 17,
 'c1qtnf12': 18,
 'ube2j2': 19,
 'scnn1d': 20,
 'acap3': 21,
 'pusl1': 22,
 'ints11': 23,
 'cptp': 24,
 'tas1r3': 25,
 'dvl1': 26,
 'mxra8': 27,
 'aurkaip1': 28,
 'ccnl2': 29,
 'mrpl20': 30,
 'ankrd65': 31,
 'tmem88b': 32,
 'vwa1': 33,
 'atad3c': 34,
 'atad3b': 35,
 'atad3a': 36,
 'tmem240': 37,
 'ssu72': 38,
 'fndc10': 39,
 'mib2': 40,
 'mmp23b': 41,
 'cdk11b': 42,
 'slc35e2b': 43,
 'cdk11a': 44,
 'nadk': 45,
 'gnb1': 46,
 'calml6': 47,
 'tmem52': 48,
 'cfap74': 49,
 'gabrd': 50,
 'prkcz': 51,
 'faap20': 52,
 'ski': 53,
 'morn1': 54,
 'rer1': 55,
 'pex10': 56,
 'plch2': 57,
 'pank4': 58,
 'hes5': 59,
 'tnfrsf14': 60,
 'prxl2b': 61,
 'mmel1': 62,
 'ttc34': 63,
 'actrt2': 64,
 'prdm16': 65,
 'arhgef16': 66,
 'megf6': 67,
 'tprg1l': 68

In [10]:
only_in_all_genes = all_genes_set - adata_genes_set

only_in_adata_genes = adata_genes_set - all_genes_set

# 输出结果
print(f"仅在 all_genes 中存在的基因数量: {len(only_in_all_genes)}")
print(f"仅在 all_genes 中存在的基因: {only_in_all_genes}")

print(f"仅在 adata_genes 中存在的基因数量: {len(only_in_adata_genes)}")
print(f"仅在 adata_genes 中存在的基因: {only_in_adata_genes}")


仅在 all_genes 中存在的基因数量: 387
仅在 all_genes 中存在的基因: {'tars1', 'eppin-wfdc6', 'ptges3l-aarsd1', 'h3-5', 'prr5-arhgap8', 'brme1', 'nme1-nme2', 'marchf2', 'myg1', 'notch2nlr', 'cibar2', 'vars1', 'senp3-eif4a1', 'marchf8', 'pwwp4', 'rars1', 'polr1g', 'znf511-prap1', 'h1-5', 'h1-0', 'arpin-ap3s2', 'rpl17-c18orf32', 'h3c6', 'eef1akmt4-ece2', 'macroh2a2', 'fam90a14p', 'cfap20dc', 'h1-6', 'mix23', 'tmdd1', 'h2ac20', 'h2bc12', 'rtel1-tnfrsf6b', 'pgap6', 'guca1anb', 'h4c8', 'irag1', 'h2bc15', 'sting1', 'marchf6', 'h2ac19', 'wash6p', 'dync2i1', 'chkb-cpt1b', 'trappc14', 'c17orf114', 'fam90a23p', 'h1-2', 'bloc1s5-txndc5', 'h4c9', 'mideas', 'h4c6', 'apoc4-apoc2', 'percc1', 'tamalin', 'ins-igf2', 'saa2-saa4', 'h3-3b', 'polr1f', 'mtarc2', 'cfap298-tcp10l', 'dusp29', 'znf625-znf20', 'h3-2', 'fam90a16p', 'odad3', 'stimate-mustn1', 'polr1h', 'cep20', 'adprs', 'dynlt2', 'linc02210-crhr1', 'st20', 'iftap', 'h3c2', 'cenatac', 'eef1e1-bloc1s5', 'h2bc18', 'nphp3-acad11', 'hars1', 'wars1', 'h2aw', 'ten1-cdk3', 'f

In [11]:
dense_adata_X = adata.X.toarray()
for i, gene in enumerate(adata_genes_lower):
    if gene in gene_to_index:
        new_data[:, gene_to_index[gene]] = dense_adata_X[:, i]
    else:
        print(f'Gene {gene} not found in all_genes list')

In [12]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split


label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(adata.obs['cell_type'])  # 预先编码标签


random_seed = 42
X_train_val, X_test, y_train_val, y_test = train_test_split(
    new_data, labels_encoded, test_size=0.15, random_state=random_seed)


X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.1765, random_state=random_seed)  # 0.1765 是为了让验证集占 15%

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

estim.model.eval()
with torch.no_grad():
    X_train_tensor = torch.tensor(X_train).float().to(device)
    X_test_tensor = torch.tensor(X_test).float().to(device)
    train_embeddings = estim.model.encoder(X_train_tensor).detach().cpu().numpy()
    test_embeddings = estim.model.encoder(X_test_tensor).detach().cpu().numpy()


cuda


In [13]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

    

    # 初始化和训练KNN分类器
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_embeddings, y_train)
    
    # 模型预测
predictions = knn.predict(test_embeddings)

    # 计算准确率和 F1 分数
accuracy = accuracy_score(y_test, predictions)
print(f"KNN Accuracy on Test Data: {accuracy}")
f1 = f1_score(y_test, predictions, average='weighted')
print(f"Weighted F1 Score: {f1}")
    
macro_f1 = f1_score(y_test, predictions, average='macro')
print(f'Macro F1 Score: {macro_f1}')

    # 计算随机猜测的准确率
class_probabilities = np.bincount(y_test) / len(y_test)
random_accuracy = np.sum(class_probabilities ** 2)
print(f"Random Guess Accuracy: {random_accuracy}")

    # 生成分类报告
report = classification_report(y_test, predictions, target_names=label_encoder.classes_)
print(report)

KNN Accuracy on Test Data: 0.7524246771879484
Weighted F1 Score: 0.7486210053020783
Macro F1 Score: 0.6469567612816001
Random Guess Accuracy: 0.05170816843656664
                          precision    recall  f1-score   support

       Astrocyte-Fibrous       0.66      0.73      0.69       659
      Astrocyte-Immature       0.50      0.57      0.53       738
  Astrocyte-Protoplasmic       0.75      0.66      0.70       478
      Cajal-Retzius cell       0.20      0.13      0.16       123
          EN-IT-Immature       0.55      0.71      0.62      2091
              EN-L2_3-IT       0.80      0.80      0.80      1792
                EN-L4-IT       0.84      0.84      0.84      3031
                EN-L5-ET       0.43      0.25      0.32       189
                EN-L5-IT       0.87      0.82      0.84      1531
              EN-L5_6-NP       0.66      0.57      0.61       284
                EN-L6-CT       0.51      0.42      0.46       588
                EN-L6-IT       0.80      0.69

In [14]:
with torch.no_grad():
    new_data_tensor = torch.tensor(new_data).float().to(device)
    SSL_embeddings = estim.model.encoder(new_data_tensor).detach().cpu().numpy()
new_adata = sc.read_h5ad(data_dir)
new_adata.obsm[f'SSL_GP_ZS_{random_seed}'] = SSL_embeddings
new_adata.uns[f'GP_ZS_y_test_{random_seed}'] = y_test
new_adata.uns[f'GP_ZS_predictions_{random_seed}'] = predictions
new_adata.uns[f'GP_ZS_target_names_{random_seed}'] = label_encoder.classes_
new_adata.write_h5ad(data_dir)

In [15]:

import pandas as pd
import os
import re

# 当前 Notebook 文件名
notebook_name = "GEX_human_neocortex_GP_mask_zero_shot_42.ipynb"

# 初始化需要打印的值
init_train_loss = train_losses[0] if 'train_losses' in globals() else None
init_val_loss = val_losses[0] if 'val_losses' in globals() else None
converged_epoch = len(train_losses) - patience if 'train_losses' in globals() else None
converged_val_loss = best_val_loss if 'best_val_loss' in globals() else None

# 打印所有所需的指标
print("Metrics Summary:")
if 'train_losses' in globals():
    print(f"init_train_loss\tinit_val_loss\tconverged_epoch\tconverged_val_loss\tmacro_f1\tweighted_f1\tmicro_f1")
    print(f"{init_train_loss:.3f}\t{init_val_loss:.3f}\t{converged_epoch}\t{converged_val_loss:.3f}\t{macro_f1:.3f}\t{f1:.3f}\t{accuracy:.3f}")
else:
    print(f"macro_f1\tweighted_f1\tmicor_f1")
    print(f"{macro_f1:.3f}\t{f1:.3f}\t{accuracy:.3f}")

# 保存结果到 CSV 文件
output_data = {
    'dataset_split_random_seed': [int(random_seed)],
    'dataset': ['GEX_human_neocortex'],
    'method': [re.search(r'human_neocortex_(.*?)_\d+', notebook_name).group(1)],
    'init_train_loss': [init_train_loss if init_train_loss is not None else ''],
    'init_val_loss': [init_val_loss if init_val_loss is not None else ''],
    'converged_epoch': [converged_epoch if converged_epoch is not None else ''],
    'converged_val_loss': [converged_val_loss if converged_val_loss is not None else ''],
    'macro_f1': [macro_f1],
    'weighted_f1': [f1],
    'micro_f1': [accuracy]
}
output_df = pd.DataFrame(output_data)

# 保存到当前目录下名为 results 的文件夹中
if not os.path.exists('results'):
    os.makedirs('results')

csv_filename = f"results/{os.path.splitext(notebook_name)[0]}_results.csv"
output_df.to_csv(csv_filename, index=False)


Metrics Summary:
macro_f1	weighted_f1	micor_f1
0.647	0.749	0.752
